# WhiteboardEnv Technical Documentation & Explainer

## Overview

The **WhiteboardEnv** is an interactive Gymnasium environment designed for studying embodied sensorimotor learning and human-AI collaboration in a writing and/or drawing task. It simulates a whiteboard with realistic physics, featuring two controllable joints:

- **Pen Joint:** Controls a pen that can draw (writing mode) or scroll the board (scrolling mode).
- **Vision Joint:** Controls the focus window (a clear region) on a blurred board, shown as a red transparent tracker.

The environment provides a composite image observation and a 4-dimensional numeric observation (reaction forces from the joints), while the action space is a hybrid of discrete selections and continuous forces.

This document explains the environment’s key features, API, and usage.

## Key Features

**Dual-Joint Control:**

- **Pen Joint:** Can be in move, writing, or scrolling mode. In writing mode, isolated dots are drawn on the board. In scrolling mode, vertical scrolling is applied to the board.
- **Vision Joint:** Controls a clear focus window over a blurred board. A red transparent square (with a border) indicates the focus area.

**Observations:**

- The agent receives a composite image (as a Pygame Surface and a corresponding NumPy array) and a 4-dimensional vector of reaction forces (from the pen and vision joints, normalized by MAX_FORCE). The explicit positions of the joints are not provided.

**Action Space:**

- **joint_select:** `Discrete(2)` (0 for pen, 1 for vision).
- **pen_mode:** `Discrete(3)` (0: move, 1: write, 2: scroll). Only used when controlling the pen.
- **pen_force:** `Box(2,)` for continuous force on the pen joint.
- **vision_force:** `Box(2,)` for continuous force on the vision joint.

**Human Interaction:**

- Human drawing: Blue dots are drawn when the mouse is used within the board.
- Scrolling: The board can be scrolled vertically via human input (mouse wheel or key events) and via the AI’s scrolling mode.

**Rendering:**

- The board is rendered as a large Pygame Surface. A viewport is extracted (based on vertical offset) and displayed in a smaller window.
- The AI pen is shown as a small black circle, and the focus window is indicated by a red transparent square with a border.

**Physics Simulation:**

- Uses Euler integration with mass and friction to update joint positions and velocities.
- Scrolling incorporates extra friction and scales the applied force to adjust the board’s vertical offset.

**Gymnasium API Compliance:**

- Implements `reset()`, `step()`, and `close()`, with defined `observation_space` and `action_space`.

## API Summary

**Constructor:** `WhiteboardEnv(render_mode="human")`
- Initializes board dimensions, creates a Pygame Surface for the board, and sets up joint states and action/observation spaces.

**reset()**
- Resets the environment and returns `(observation, info)`, where the observation includes:
  - `image_surface`: the composite image (Pygame Surface),
  - `image_array`: the corresponding NumPy array,
  - `numeric`: a 4D vector of reaction forces.

**get_state()**
- Returns the current observation.

**step(action)**
- Processes an action (provided as a dictionary) and updates joint states and board state.
  - For pen control, uses `pen_mode` to determine behavior (move, write, or scroll).
  - For vision control, applies the provided vision forces.
- Returns `(observation, reward, terminated, truncated, info)`.

**render()**
- Extracts a viewport from the board based on a vertical offset and displays it using Pygame.
  - Overlays indicators for the human pen, AI pen, and AI focus window.

**close()**
- Closes the Pygame display and cleans up resources.

## Usage Example

Below is an example showing how to initialize and run the environment interactively.

```python
import pygame
import gymnasium as gym
from whiteboard_env import WhiteboardEnv  # Ensure the environment is in this module

# Initialize the environment
env = WhiteboardEnv(render_mode="human")
observation, info = env.reset()

# Main interactive loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        # Additional event handling for human input goes here

    # Sample a random action (or use your RL agent's policy)
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    # Render the environment
    env.render()

env.close()
```

This example demonstrates the basic workflow: reset, interact via step, render, and finally close the environment.

## Design Considerations

- **Observation Design:**
  - The agent does **not** receive explicit pen or vision positions. Instead, it must infer these from the composite image and reaction forces, encouraging more robust sensorimotor learning.

- **Action Abstraction:**
  - The hybrid action space combines discrete decisions (which joint to control, and for the pen, what mode to use) with continuous forces, reflecting the complexity of real-world motor control.

- **Realistic Physics:**
  - Joint updates incorporate mass, friction, and extra factors (for scrolling) to produce realistic movement.

- **Human-AI Interaction:**
  - The environment supports concurrent human drawing (via mouse) and AI control, enabling studies of collaboration or competition.

- **Extensibility:**
  - The modular design makes it easy to add more sophisticated reward signals, additional sensor modalities, or alternative control algorithms.

## Conclusion

The **WhiteboardEnv** is a rich, interactive environment for studying sensorimotor learning and human-AI collaboration in a drawing task. Its realistic physics, hybrid action space, and multi-modal observations make it a valuable tool for reinforcement learning research.

Feel free to explore, modify, and extend this environment to suit your experimental needs.

In [ ]:
# Sample Usage
import pygame
from whiteboard_env import WhiteboardEnv  # Adjust the import path as needed

env = WhiteboardEnv(render_mode="human")
obs, info = env.reset()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Here, you can replace the random action with your agent's policy
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)

    # Optionally, process reward or update your agent
    if reward != 0:
        print("Reward event:", reward)

    env.render()
    pygame.time.Clock().tick(60)

env.close()
